In [1]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dict = {
    "../../data/adult-census-income.csv" : "income", 
    "../../data/measuring-hate-speech.csv" : "insult",
    "../../data/credit-card-clients.csv" : "default.payment.next.month",
    "../../data/diabetes_prediction_dataset.csv" : "diabetes",
    "../../data/hospital_readmissions_30k.csv" : "readmitted_30_days",
    "../../data/Reasoning_Patterns_AI_Hiring_Bias_SEA.csv" : "Is Bias Likely? (Yes/No)",
}

In [3]:
"""
from kaggle.api.kaggle_api_extended import KaggleApi 

api = KaggleApi()
api.authenticate()
# Download latest version
path = api.dataset_download_file("danofer/law-school-admissions-bar-passage", "bar_pass_prediction.csv")

print("Path to dataset files:", path)
"""

'\nfrom kaggle.api.kaggle_api_extended import KaggleApi \n\napi = KaggleApi()\napi.authenticate()\n# Download latest version\npath = api.dataset_download_file("danofer/law-school-admissions-bar-passage", "bar_pass_prediction.csv")\n\nprint("Path to dataset files:", path)\n'

In [4]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

for index, data in enumerate(DATASETS.items()):
    data_link = data[0]
    info = data[1]
    if index <= 50:
        continue
    print(data_link)
    if data_link in ["https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data",
                     "https://www.kaggle.com/datasets/fleshmetal/records-a-comprehensive-music-metadata-dataset",
                     "https://www.kaggle.com/datasets/sudipde25/lifespan-calorie-burn-dynamics"]:
        continue
    target_column = info["target_column"]
    nr_file = info["no_dataset"]

    if data_link == "https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech":
        ingestor_factory = IngestorFactory(data_link, nr_file, True)
    else:
        ingestor_factory = IngestorFactory(data_link, nr_file, False)
    ingestor = ingestor_factory.create()
    data = ingestor.load_data()

    preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
    preprocessor = preprocessor_factory.create()
    processed_data_dict = preprocessor.process_data(ohe=True)

https://www.kaggle.com/datasets/stealthtechnologies/basketball-players-dataset
data.csv
Dataset URL: https://www.kaggle.com/datasets/stealthtechnologies/basketball-players-dataset
../../data/kaggle_temp/data.csv was deleted.
dropping Category
Base_Income
['21–29', '30–30', '31–32', '33–33', '34–34', '35–36', '37–37', '38–40', '41–42', '43–47']
https://www.kaggle.com/datasets/miadul/parkinsons-disease-synthetic-dataset
parkinsons_disease_dataset.csv
Dataset URL: https://www.kaggle.com/datasets/miadul/parkinsons-disease-synthetic-dataset
../../data/kaggle_temp/parkinsons_disease_dataset.csv was deleted.
Age
['40–43', '44–48', '49–52', '53–57', '58–61', '62–66', '67–70', '71–75', '76–80', '81–84']
https://www.kaggle.com/datasets/youssefelebiary/1985-auto-imports-dataset-uci-ml-repository
data.csv
Dataset URL: https://www.kaggle.com/datasets/youssefelebiary/1985-auto-imports-dataset-uci-ml-repository
../../data/kaggle_temp/data.csv was deleted.
https://www.kaggle.com/datasets/ahmethoso/wpp

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

data = pd.read_csv("../../data/adult.csv")
target_column = "income"

#data = pd.read_csv("../../data/measuring-hate-speech.csv")
#target_column = "insult"

#data = pd.read_csv("../../data/credit-card-clients.csv")
#target_column = "default.payment.next.month"

#data = pd.read_csv("../../data/diabetes_prediction_dataset.csv")
#target_column = "diabetes"

#data = pd.read_csv("../../data/Reasoning_Patterns_AI_Hiring_Bias_SEA.csv")
#target_column = "Is Bias Likely? (Yes/No)"


c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def run_preprocessor_checks():
    methods = ["data quality", "fairness"]
    ohe_flags = [False, True]

    for index, data in enumerate(DATASETS.items()):
        data_link = data[0]
        info = data[1]
        if data_link in ["https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data",
                        "https://www.kaggle.com/datasets/fleshmetal/records-a-comprehensive-music-metadata-dataset",
                        "https://www.kaggle.com/datasets/sudipde25/lifespan-calorie-burn-dynamics"]:
            continue
        target_column = info["target_column"]
        nr_file = info["no_dataset"]
        # 1) CSV laden
        ingestor_factory = IngestorFactory(data_link, nr_file, False)
        ingestor = ingestor_factory.create()
        df = ingestor.load_data()
        print(f"\n==> Testing {data_link!r}  target={target_column}")

        for method in methods:
            for ohe in ohe_flags:
                label = f"  [{method:12s} | ohe={ohe}]"
                try:
                    # 2) Factory + Preprocessor
                    factory = PreprocessorFactory(df, method, target_column)
                    pre     = factory.create()

                    # 3) Aufruf process_data
                    result = pre.process_data(ohe=ohe)

                    # 4) Struktur prüfen
                    if not isinstance(result, tuple):
                        raise AssertionError(f"{label} ➞ returned no tuple")
                    if len(result) != 6:
                        raise AssertionError(f"{label} ➞ expected 6 items, got {len(result)}")

                    df_tr, num_cols, cat_cols, text_feats, sens_cols, tgt_col = result

                    # 5a) DataFrame-Typ und Zeilenzahl
                    if not isinstance(df_tr, pd.DataFrame):
                        raise AssertionError(f"{label} ➞ first item not a DataFrame")
                    if df_tr.shape[0] != df.shape[0]:
                        raise AssertionError(f"{label} ➞ row count changed {df_tr.shape[0]} vs {df.shape[0]}")

                    # 5b) Target-Spalte vorhanden
                    if target_column not in df_tr.columns:
                        raise AssertionError(f"{label} ➞ target column '{target_column}' missing")

                    # 6) Die vier Listen müssen Listen sein
                    for name, lst in [
                        ("numeric_columns",    num_cols),
                        ("categorical_columns",cat_cols),
                        ("text_features",      text_feats),
                        ("sensitive_columns",  sens_cols)
                    ]:
                        if not isinstance(lst, list):
                            raise AssertionError(f"{label} ➞ '{name}' is not a list")

                    # 7) target_column-Return
                    if not isinstance(tgt_col, str) or tgt_col != target_column:
                        raise AssertionError(f"{label} ➞ returned target '{tgt_col}'")

                    print(f"{label} ✓")

                except Exception as e:
                    print(f"{label} ✗ ERROR: {e}")
                    # Wenn Du an erster Fehler-Abbruch willst, kannst Du hier:
                    # raise
    print("\nAll checks done.")


run_preprocessor_checks()

Creating CSV from Arrow format: 100%|██████████| 33/33 [00:00<00:00, 373.44ba/s]



==> Testing 'https://huggingface.co/datasets/scikit-learn/adult-census-income'  target=income
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
student_habits_performance.csv
Dataset URL: https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance
../../data/kaggle_temp/student_habits_performance.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance'  target=exam_score
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
student_info.csv
Dataset URL: https://www.kaggle.com/datasets/therohithanand/student-academic-performance-dataset
../../data/kaggle_temp/student_info.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/therohithanand/student-academic-performance-dataset'  target=final_result
  [data quality | ohe=False] ✓
  [data quality | ohe=True] 

Creating CSV from Arrow format: 100%|██████████| 100/100 [00:00<00:00, 564.95ba/s]



==> Testing 'https://huggingface.co/datasets/marianeft/diabetes_prediction_dataset'  target=diabetes
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓


Creating CSV from Arrow format: 100%|██████████| 30/30 [00:00<00:00, 197.38ba/s]


...

tested.csv
Dataset URL: https://www.kaggle.com/datasets/brendan45774/test-file
../../data/kaggle_temp/tested.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/brendan45774/test-file'  target=Survived
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
speeddating.csv
Dataset URL: https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating
../../data/kaggle_temp/speeddating.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating/data'  target=match
dropping has_null
  [data quality | ohe=False] ✓
dropping has_null
  [data quality | ohe=True] ✓
dropping has_null
  [fairness     | ohe=False] ✓
dropping has_null
  [fairness     | ohe=True] ✓
451
<class 'str'>

==> Testing 'https://www.openml.org/search?type=data&sort=runs&status=active&id=451'  target=Leaving_Certificate
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [f

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(


fertility.csv
Dataset URL: https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set
../../data/kaggle_temp/fertility.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set'  target=Diagnosis
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
physics_teacher_jobs_linkedin.csv
Dataset URL: https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin
../../data/kaggle_temp/physics_teacher_jobs_linkedin.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin'  target=contractType
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
Public Vs Private Universties.csv
Dataset URL: https://www.kaggle.com/datasets/amosshehzad/public-vs-privat-schools-statistical-analysis
../../data/kaggle_temp/Public Vs Private Universties.csv was deleted.

==>

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\ingestion\ingestorFactory.py:380: DtypeWarning: Columns (20,50,161,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(final_csv)



==> Testing 'https://www.kaggle.com/datasets/threnjen/portland-housing-prices-sales-jul-2020-jul-2021'  target=price
dropping priceHistory/0/showCountyLink
dropping priceHistory/1/showCountyLink
dropping priceHistory/2/showCountyLink
dropping priceHistory/3/showCountyLink
dropping priceHistory/4/showCountyLink
dropping priceHistory/5/showCountyLink
dropping priceHistory/6/showCountyLink
dropping priceHistory/7/showCountyLink
dropping priceHistory/8/showCountyLink
dropping priceHistory/9/showCountyLink
dropping priceHistory/10/showCountyLink
dropping resoFacts/hasAdditionalParcels
dropping resoFacts/hasOpenParking
dropping schools/2/level
  [data quality | ohe=False] ✓
dropping priceHistory/0/showCountyLink
dropping priceHistory/1/showCountyLink
dropping priceHistory/2/showCountyLink
dropping priceHistory/3/showCountyLink
dropping priceHistory/4/showCountyLink
dropping priceHistory/5/showCountyLink
dropping priceHistory/6/showCountyLink
dropping priceHistory/7/showCountyLink
dropping p

In [ ]:
preprocessor_factory = PreprocessorFactory(data, "data quality", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=False)
processed_data_dict[0]

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,age,education.num,capital.loss,fnlwgt,capital.gain,hours.per.week,income
0,?,HS-grad,Widowed,?,Not-in-family,White,Female,United-States,90,9,4356,77053,0,40,<=50K
1,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,United-States,82,9,4356,132870,0,18,<=50K
2,?,Some-college,Widowed,?,Unmarried,Black,Female,United-States,66,10,4356,186061,0,40,<=50K
3,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States,54,4,3900,140359,0,40,<=50K
4,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,United-States,41,10,3900,264663,0,40,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,Private,Some-college,Never-married,Protective-serv,Not-in-family,White,Male,United-States,22,10,0,310152,0,40,<=50K
32557,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,27,12,0,257302,0,38,<=50K
32558,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,40,9,0,154374,0,40,>50K
32559,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,58,9,0,151910,0,40,<=50K


In [ ]:
for column in processed_data_dict[4]:
    if column not in processed_data_dict[2]:
        raise TypeError(f"{column} not categorical!")

In [2]:
data

,Êcommunityname,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39,5320,1,11980,3.10,1.37,91.78,6.50,...,14,114.85,138,1132.08,16,131.26,2,16.41,41.02,1394.59
1,Marpletownship,PA,45,47616,1,23123,2.82,0.80,95.57,3.44,...,57,242.37,376,1598.78,26,110.55,1,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,,1,29344,2.43,0.74,94.33,3.43,...,274,758.14,1797,4972.19,136,376.3,22,60.87,218.59,6167.51
3,Gloversvillecity,NY,35,29443,1,16656,2.40,1.70,97.35,0.50,...,225,1301.78,716,4142.56,47,271.93,NaN,,306.64,
4,Bemidjicity,MN,7,5068,1,11245,2.76,0.53,89.16,1.17,...,91,728.93,1060,8490.87,91,728.93,5,40.05,NaN,9988.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,Mercedcity,CA,NaN,,10,56216,3.07,6.87,61.68,15.23,...,1376,2268.72,2563,4225.82,489,806.25,34,56.06,545.75,7356.84
2211,Pinevillecity,LA,NaN,,10,12251,2.68,21.18,76.65,1.52,...,104,860.43,574,4748.9,24,198.56,2,16.55,124.1,5824.44
2212,Yucaipacity,CA,NaN,,10,32824,2.46,0.52,92.62,0.98,...,628,1709.26,895,2435.97,179,487.19,8,21.77,353.83,4654.2
2213,Beevillecity,TX,NaN,,10,13547,2.89,3.37,69.91,0.90,...,192,1508.01,474,3722.9,13,102.1,1,7.85,691.17,5340.87


In [3]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

data_link = "https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech"
nr_file = 1
target_column = "hate_speech_score"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

Creating CSV from Arrow format: 100%|██████████| 136/136 [00:02<00:00, 49.16ba/s]


dropping annotator_ideology_extremeley_liberal
Hier


,target_race_asian_False,target_race_asian_True,target_race_black_False,target_race_black_True,target_race_latinx_False,target_race_latinx_True,target_race_middle_eastern_False,target_race_middle_eastern_True,target_race_native_american_False,target_race_native_american_True,...,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,text_svd_0,text_svd_1,hate_speech_score
0,0,1,0,1,0,1,0,1,0,1,...,-0.462431,1.399639,0.782464,0.206499,1.287673,0.462446,-1.871698,0.302750,-0.143037,1
1,1,0,1,0,1,0,1,0,1,0,...,-0.146203,-0.852191,-0.685419,3.297695,-1.638428,-1.103696,-1.899148,0.127300,-0.043538,0
2,1,0,1,0,1,0,1,0,1,0,...,-0.272694,-0.370765,-0.954186,0.065990,0.574905,-0.620319,-0.363524,0.250193,-0.131525,6
3,1,0,1,0,1,0,1,0,1,0,...,-0.462431,-0.463945,1.816184,-0.027683,0.649933,-0.233617,-0.115378,0.265575,-0.109352,5
4,1,0,1,0,1,0,1,0,1,0,...,-0.567840,-0.541594,-1.078233,1.002716,-0.212892,0.017739,0.145724,0.205864,-0.011501,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,1,0,1,0,1,0,1,0,1,0,...,-1.832751,-1.349147,-2.049930,1.845770,-0.100349,-0.813670,-0.083858,0.218872,-0.095433,0
135552,1,0,1,0,1,0,0,1,1,0,...,0.570580,-0.852191,-1.098907,0.253335,0.424848,2.241274,-1.617570,0.148161,-0.062025,0
135553,1,0,1,0,1,0,1,0,1,0,...,0.170025,0.483377,0.368976,0.347008,1.700329,0.114414,-0.828640,0.158163,-0.057673,2
135554,1,0,1,0,1,0,1,0,1,0,...,0.570580,-0.852191,-0.954186,0.253335,-0.025321,-0.465638,-1.617570,0.148161,-0.062025,0


In [4]:
processed_data_dict[0].shape

(2215, 1616)

In [2]:
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
import sys, os
sys.path.append(os.path.abspath('..')) 
from datasets_dict import DATASETS
from ingestion.ingestorFactory import IngestorFactory

data = pd.read_csv("../../data/adult.csv")
target_column = "income"
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

,age_0-9,age_10-19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,age_80-89,age_90-99,...,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income
0,0,0,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,-1.118058,-0.427466,-0.251406,4.951570,-0.032700,0
1,0,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,-0.558414,-0.427466,-0.251406,4.951570,-1.875394,0
2,0,0,0,0,0,0,1,0,0,0,...,0.0,0.0,1.0,0.0,-0.025099,-0.034925,-0.251406,4.951570,-0.032700,0
3,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,-0.483326,-2.390174,-0.251406,4.951570,-0.032700,0
4,0,0,0,1,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.762998,-0.034925,-0.251406,4.951570,-0.032700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.219089,-0.034925,-0.251406,-0.218828,-0.032700,0
32557,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.689193,0.750158,-0.251406,-0.218828,-0.200218,0
32558,0,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,-0.342806,-0.427466,-0.251406,-0.218828,-0.032700,1
32559,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,-0.367511,-0.427466,-0.251406,-0.218828,-0.032700,0


In [ ]:
for column in processed_data_dict[4]:
    if column not in processed_data_dict[2]:
        raise TypeError(f"{column} not categorical!")

In [2]:
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=False)
processed_data_dict[0]

,age,marital.status,occupation,race,sex,native.country,workclass,education,relationship,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income
0,90-99,Widowed,?,White,Female,United-States,?,HS-grad,Not-in-family,-1.118058,-0.427466,-0.251406,4.951570,-0.032700,0
1,80-89,Widowed,Exec-managerial,White,Female,United-States,Private,HS-grad,Not-in-family,-0.558414,-0.427466,-0.251406,4.951570,-1.875394,0
2,60-69,Widowed,?,Black,Female,United-States,?,Some-college,Unmarried,-0.025099,-0.034925,-0.251406,4.951570,-0.032700,0
3,50-59,Divorced,Machine-op-inspct,White,Female,United-States,Private,7th-8th,Unmarried,-0.483326,-2.390174,-0.251406,4.951570,-0.032700,0
4,30-39,Separated,Prof-specialty,White,Female,United-States,Private,Some-college,Own-child,0.762998,-0.034925,-0.251406,4.951570,-0.032700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0-9,Never-married,Protective-serv,White,Male,United-States,Private,Some-college,Not-in-family,1.219089,-0.034925,-0.251406,-0.218828,-0.032700,0
32557,10-19,Married-civ-spouse,Tech-support,White,Female,United-States,Private,Assoc-acdm,Wife,0.689193,0.750158,-0.251406,-0.218828,-0.200218,0
32558,30-39,Married-civ-spouse,Machine-op-inspct,White,Male,United-States,Private,HS-grad,Husband,-0.342806,-0.427466,-0.251406,-0.218828,-0.032700,1
32559,50-59,Widowed,Adm-clerical,White,Female,United-States,Private,HS-grad,Unmarried,-0.367511,-0.427466,-0.251406,-0.218828,-0.032700,0


In [ ]:
for column in processed_data_dict[4]:
    if column not in processed_data_dict[2]:
        raise TypeError(f"{column} not categorical!")

In [ ]:
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

dropping annotator_ideology_extremeley_liberal
dropping empty_col
dropping const_col


,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,...,violence,attack_defend,hate_speech_score,outfitms,std_err,annotator_outfitms,hypothesis,text_svd_0,text_svd_1,insult
0,True,True,True,True,True,True,True,False,True,False,...,-0.781782,-2.355244,-1.418621,1.399639,0.206499,0.462446,-1.871698,0.302746,-0.144076,0.00–1.00
1,False,False,False,False,False,False,False,False,False,False,...,-0.781782,-0.561450,-2.285782,-0.852191,3.297695,-1.103696,-1.899148,0.127298,-0.045128,0.00–1.00
2,False,False,False,False,False,False,False,False,False,False,...,-0.781782,1.232343,0.392216,-0.370765,0.065990,-0.620319,-0.363524,0.250195,-0.128565,3.00–4.00
3,False,False,False,False,False,False,False,False,False,False,...,-0.781782,0.335446,0.349708,-0.463945,-0.027683,-0.233617,-0.115378,0.265577,-0.108116,1.00–2.00
4,False,False,False,False,False,False,False,False,False,False,...,2.190647,0.335446,0.893809,-0.541594,1.002716,0.017739,0.145724,0.205864,-0.013201,3.00–4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135551,False,False,False,False,False,False,False,False,False,False,...,-0.781782,-0.561450,-1.835198,-1.349147,1.845770,-0.813670,-0.083858,0.218873,-0.098694,0.00–1.00
135552,False,False,False,True,False,False,False,False,True,False,...,-0.781782,-1.458347,-1.631160,-0.852191,0.253335,2.241274,-1.617570,0.148162,-0.063407,0.00–1.00
135553,False,False,False,False,False,False,False,False,False,False,...,-0.781782,-1.458347,-0.819259,0.483377,0.347008,0.114414,-0.828640,0.158162,-0.056794,0.00–1.00
135554,False,False,False,False,False,False,False,False,False,False,...,-0.781782,-0.561450,-1.631160,-0.852191,0.253335,-0.465638,-1.617570,0.148162,-0.063407,0.00–1.00


In [ ]:
print("sensitive columns")
processed_data_dict[4]

sensitive columns


['target_race_asian',
 'target_race_black',
 'target_race_latinx',
 'target_race_middle_eastern',
 'target_race_native_american',
 'target_race_pacific_islander',
 'target_race_white',
 'target_race_other',
 'target_race',
 'target_religion_atheist',
 'target_religion_buddhist',
 'target_religion_christian',
 'target_religion_hindu',
 'target_religion_jewish',
 'target_religion_mormon',
 'target_religion_muslim',
 'target_religion_other',
 'target_religion',
 'target_gender_men',
 'target_gender_non_binary',
 'target_gender_transgender_men',
 'target_gender_transgender_unspecified',
 'target_gender_transgender_women',
 'target_gender_women',
 'target_gender_other',
 'target_gender',
 'target_sexuality_bisexual',
 'target_sexuality_gay',
 'target_sexuality_lesbian',
 'target_sexuality_straight',
 'target_sexuality_other',
 'target_sexuality',
 'target_age_children',
 'target_age_teenagers',
 'target_age_young_adults',
 'target_age_middle_aged',
 'target_age_seniors',
 'target_age_other'

In [ ]:
for column in processed_data_dict[4]:
    if column not in processed_data_dict[2]:
        raise TypeError(f"{column} not categorical!")

In [ ]:
from preprocessing import Preprocessor as DQP
numeric_columns, categorical_columns = DQP(data, target_column="").receive_categorized_columns()

dropping annotator_ideology_extremeley_liberal
dropping empty_col
dropping const_col


In [ ]:
numeric_columns

['comment_id',
 'platform',
 'respect',
 'humiliate',
 'dehumanize',
 'genocide',
 'hatespeech',
 'infitms',
 'annotator_severity',
 'annotator_infitms',
 'hypothesis']

In [ ]:
categorical_columns

['annotator_gender',
 'annotator_trans',
 'annotator_educ',
 'annotator_income',
 'annotator_ideology',
 'annotator_id',
 'sentiment',
 'insult',
 'status',
 'violence',
 'attack_defend',
 'hate_speech_score',
 'outfitms',
 'std_err',
 'annotator_outfitms',
 'annotator_age',
 'target_race_asian',
 'target_race_black',
 'target_race_latinx',
 'target_race_middle_eastern',
 'target_race_native_american',
 'target_race_pacific_islander',
 'target_race_white',
 'target_race_other',
 'target_race',
 'target_religion_atheist',
 'target_religion_buddhist',
 'target_religion_christian',
 'target_religion_hindu',
 'target_religion_jewish',
 'target_religion_mormon',
 'target_religion_muslim',
 'target_religion_other',
 'target_religion',
 'target_gender_men',
 'target_gender_non_binary',
 'target_gender_transgender_men',
 'target_gender_transgender_unspecified',
 'target_gender_transgender_women',
 'target_gender_women',
 'target_gender_other',
 'target_gender',
 'target_sexuality_bisexual',
 '

In [ ]:
dict = DQP(data, target_column="").receive_number_of_columns()
dict

dropping annotator_ideology_extremeley_liberal
dropping empty_col
dropping const_col
dropping annotator_ideology_extremeley_liberal
dropping empty_col
dropping const_col


{'numeric_columns': 11, 'categorical_columns': 97, 'text_columns': 1}